# Recitation 4
Topics: Anonymous Functions, Semantics, Scoping

## Anonymous functions

Syntax:
```
Scala:  (param1, param2) => expression
Python: lambda param1, param2: expression
Java:   (param1, param2) -> expression
```

### Exercise: Anonymous func intro
Make anonymous functions that perform the requested operations. (Ok, we're putting them in variables, so they're not quite "anonymous")

**WARNING**: `return` doesn't do what you expect in anonymous functions: https://stackoverflow.com/questions/17754976/scala-return-statements-in-anonymous-functions

In [1]:
// Returns true if input is 1, false otherwise
val is_one = (i: Int) => if (i == 1) true else false
assert(is_one(1))
assert(!is_one(2))

// new: pattern matching
// Returns true if input is 1, false otherwise *using patterrn matching*
val is_one_pattern = (i: Int) => i match{
    case 1 => true
    case _ => false
} 
assert(is_one_pattern(1))
assert(!is_one_pattern(2))

// new: multi-param
// Returns the addition of the inputs
val add = (i: Int, j: Int) => i + j
assert(add(1, 2) == 3)

// new: return funcs
// Returns a function that adds the given number to its input
val make_adder = (i: Int) => (f => j+i)
assert(make_adder(1)(2) == 3)

// new: take funcs
// Takes a function and applies it to 3
val call_on_3: (Int => Int) => Int = ???
assert(call_on_3((x) => x + 5) == 8)

// new: statements
// Any anonymous function with a loop in it
val func_with_loop: (Int) => Unit = ???

cmd1.sc:21: missing parameter type
val make_adder = (i: Int) => (f => j+i)
                              ^cmd1.sc:21: not found: value j
val make_adder = (i: Int) => (f => j+i)
                                   ^Compilation Failed

: 

### Exercise: Mini interpreter
Implement the function below to apply the operations, from left to right of the list to the input by passing an anonymous function to fold

In [8]:
sealed trait Operation
case class Plus(n: Double) extends Operation
case class Minus(n: Double) extends Operation
case class Pow(n: Double) extends Operation
case class NestedOps(ops: List[Operation]) extends Operation
case object Sin extends Operation

def perform_ops(init: Double, steps: List[Operation]): Double = {
    steps.foldLeft(init)(
        (acc, op) => op match {
            case Plus(n) => acc + n
            case Pow(n) => scala.math.pow(acc, n)
            case Minus(n) => acc - n
            case NestedOps(ops) => perform_ops(acc, ops)
            case Sin => scala.math.sin(acc)
        }
    )
}

assert(perform_ops(5, List(Plus(5), Pow(2), NestedOps(List(Minus(3))))) == 97)

defined trait Operation
defined class Plus
defined class Minus
defined class Pow
defined class NestedOps
defined object Sin
defined function perform_ops

### Exercise: Operartional semantics
Write inference rules to define `Plus` and `NestedOps` (on paper).

### Exercise: Sort
Don't worry, you don't have to write quick-sort. Use Scala's built in `sortWith` to sort food in the requested ways with anonymous functions.

```scala
// Method of list class
def sortWith(lt: (A, A) ⇒ Boolean): List[A]
```

In [9]:
case class Food(val calories: Double, val price: Double) // Like a sealed trait with fields

val donut = Food(300, 1.50)
val goat_cheese_spinach_kale_and_soy_infused_kombucha = Food(-50, 1000)
val buffalo_chicken_pizza = Food(500, 20)

val menu = List(donut, goat_cheese_spinach_kale_and_soy_infused_kombucha, buffalo_chicken_pizza)

val sorted_by_calories = menu.sortWith(_.calories < _.calories)
val sorted_by_price = menu.sortWith(_.price < _.price)

assert(sorted_by_calories == List(goat_cheese_spinach_kale_and_soy_infused_kombucha, donut, buffalo_chicken_pizza))
assert(sorted_by_price == List(donut, buffalo_chicken_pizza, goat_cheese_spinach_kale_and_soy_infused_kombucha))

defined class Food
donut: Food = Food(300.0, 1.5)
goat_cheese_spinach_kale_and_soy_infused_kombucha: Food = Food(-50.0, 1000.0)
buffalo_chicken_pizza: Food = Food(500.0, 20.0)
menu: List[Food] = List(
  Food(300.0, 1.5),
  Food(-50.0, 1000.0),
  Food(500.0, 20.0)
)
sorted_by_calories: List[Food] = List(
  Food(-50.0, 1000.0),
  Food(300.0, 1.5),
  Food(500.0, 20.0)
)
sorted_by_price: List[Food] = List(
  Food(300.0, 1.5),
  Food(500.0, 20.0),
  Food(-50.0, 1000.0)
)

## Scoping
Declarations and bindings and scopes, oh my!

###  Exercise: Lettuce concrete syntax scoping
For each variable use below, match each use with its declaration.

```
let a = 5 in
    let b = a in
        let a = b in a + b
```

```
let a = function(a)
            let a = a in
                a(5) + 4
    in
        function(b)
            let a = a(let a = 5 in a) in
                a(a(b))
```

###  Exercise: Lettuce abstract syntax scoping
For each variable use below, match each use with its declaration.

```
Let("x",
    Let("f",
        FunDef("y"
               Plus(Ident("x", Ident("y"))))
        FunCall(Ident("f"), Const(3))),
    Let("y",
        Ident("x"),
        FunCall(Ident("f"), Ident("y")))
```

###  Exercise: Scala concrete syntax scoping
For each variable use below, match each use with its declaration.

In [10]:
   val x = 5
   val y = x + 3
   val f2: (Int => Int) => Int = (y) => {
       y(x)
   }

   val f3: ((Int => Int) => Int) => Int =
       (f3) => f3((f3) => f3)
   f3(f2)

x: Int = 5
y: Int = 8
f2: Int => Int => Int = ammonite.$sess.cmd9$Helper$$Lambda$3497/0x0000000800e52840@6f02bc53
f3: Int => Int => Int => Int = ammonite.$sess.cmd9$Helper$$Lambda$3498/0x0000000800e53840@56ea772f
res9_4: Int = 5

In [11]:
   val x = 5
   x match {
       case x if x == x => {
           val x = 4
           x
       }
       case y if x == y =>
           ((x: Int) => y + x)(x)
       // Extra credit...
       case `x` => x match {
           case x @ `x` => x
       }
   }

x: Int = 5
res10_1: Int = 4

### End
* Project
    * Running
    * Testing
    * Zipping
* Assignment 4
* Assignment 5
* Quiz